In [273]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
import math

In [202]:
data = pd.read_csv('Video_Games.csv'); data

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Rapala Trophies,PSP,2006.0,Sports,Activision,0.04,0.00,0.00,0.00,NaN,NaN,NaN,NaN,Sand Grain Studios,E
1,New Super Mario Bros. U,WiiU,2012.0,Platform,Nintendo,2.30,1.34,1.27,0.32,84.0,70.0,8.1,733.0,Nintendo,E
2,Robots,PS2,2005.0,Action,Vivendi Games,0.18,0.14,0.00,0.05,53.0,6.0,6.0,8.0,Eurocom Entertainment Software,E
3,Hamster Club 3,GBA,2002.0,Simulation,Jorudan,0.00,0.00,0.35,0.01,NaN,NaN,NaN,NaN,NaN,NaN
4,Formula 1 06,PS2,2006.0,Racing,Sony Computer Entertainment,0.00,0.00,0.04,0.00,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11698,Call of Duty: Black Ops,X360,2010.0,Shooter,Activision,9.70,3.68,0.11,1.13,87.0,89.0,6.3,1454.0,Treyarch,M
11699,Resistance 3,PS3,2011.0,Shooter,Sony Computer Entertainment,0.64,0.57,0.05,0.22,83.0,91.0,7.8,543.0,Insomniac Games,M
11700,Ready 2 Rumble Revolution,Wii,2009.0,Sports,Atari,0.15,0.00,0.00,0.01,37.0,31.0,NaN,NaN,10tacle Studios,T
11701,Monster Hunter Frontier Online: Forward 2,X360,2011.0,Role-Playing,Capcom,0.00,0.00,0.03,0.00,NaN,NaN,NaN,NaN,NaN,NaN


In [203]:
data.shape

(11703, 15)

In [204]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11703 entries, 0 to 11702
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             11702 non-null  object 
 1   Platform         11703 non-null  object 
 2   Year_of_Release  11508 non-null  float64
 3   Genre            11702 non-null  object 
 4   Publisher        11665 non-null  object 
 5   NA_Sales         11703 non-null  float64
 6   EU_Sales         11703 non-null  float64
 7   JP_Sales         11703 non-null  float64
 8   Other_Sales      11703 non-null  float64
 9   Critic_Score     5676 non-null   float64
 10  Critic_Count     5676 non-null   float64
 11  User_Score       5319 non-null   float64
 12  User_Count       5319 non-null   float64
 13  Developer        7064 non-null   object 
 14  Rating           6957 non-null   object 
dtypes: float64(9), object(6)
memory usage: 1.3+ MB


In [205]:
data.describe()

,Year_of_Release,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Critic_Score,Critic_Count,User_Score,User_Count
count,11508.000000,11703.000000,11703.000000,11703.000000,11703.000000,5676.000000,5676.000000,5319.000000,5319.000000
mean,2006.519465,0.269527,0.147226,0.079129,0.047640,69.057787,26.496124,7.132995,158.593345
std,5.922082,0.887505,0.527578,0.319529,0.189604,13.959621,18.998469,1.492816,515.113357
min,1980.000000,0.000000,0.000000,0.000000,0.000000,17.000000,4.000000,0.000000,4.000000
25%,2003.000000,0.000000,0.000000,0.000000,0.000000,60.000000,12.000000,6.400000,10.000000
50%,2007.000000,0.080000,0.020000,0.000000,0.010000,71.000000,22.000000,7.500000,25.000000
75%,2010.000000,0.240000,0.110000,0.040000,0.030000,80.000000,37.000000,8.200000,81.000000
max,2020.000000,41.360000,28.960000,10.220000,10.570000,98.000000,107.000000,9.600000,9629.000000


In [206]:
data['Rating'].value_counts()

E       2810
T       2072
M       1068
E10+     996
EC         6
K-A        3
AO         1
RP         1
Name: Rating, dtype: int64

In [207]:
for i in data.columns[2:]:
    print(i, pd.unique(data[i]))

Year_of_Release [2006. 2012. 2005. 2002. 2009. 2003. 2007. 1996. 2015. 2008. 2004. 2013.
 2011. 2014. 1994. 1998.   nan 2010. 1982. 2001. 1993. 2000. 1980. 2016.
 1995. 1999. 1992. 1997. 1987. 1984. 1991. 1981. 1990. 1983. 1989. 1988.
 2017. 1986. 2020. 1985.]
Genre ['Sports' 'Platform' 'Action' 'Simulation' 'Racing' 'Role-Playing'
 'Shooter' 'Fighting' 'Misc' 'Strategy' 'Adventure' 'Puzzle' nan]
Publisher ['Activision' 'Nintendo' 'Vivendi Games' 'Jorudan'
 'Sony Computer Entertainment' '505 Games' 'CCP' 'THQ' 'Electronic Arts'
 'Banpresto' 'Ubisoft' 'Konami Digital Entertainment'
 'Brash Entertainment' 'Enix Corporation' 'Sega' 'Nippon Ichi Software'
 'D3Publisher' 'Square Enix' 'Nordic Games' 'Universal Interactive'
 'Oxygen Interactive' 'Namco Bandai Games' 'Titus' nan 'Atari' 'LucasArts'
 'Take-Two Interactive' 'NetRevo' 'Activision Value' 'SquareSoft'
 'From Software' 'Capcom' 'UFO Interactive' 'Infogrames' 'Tecmo Koei'
 'Warner Bros. Interactive Entertainment' 'Benesse'
 'System 

In [208]:
for i in data.columns:
    print(i, data[i].isna().sum())

Name 1
Platform 0
Year_of_Release 195
Genre 1
Publisher 38
NA_Sales 0
EU_Sales 0
JP_Sales 0
Other_Sales 0
Critic_Score 6027
Critic_Count 6027
User_Score 6384
User_Count 6384
Developer 4639
Rating 4746


In [209]:
data.drop(columns = ['Year_of_Release', 'Developer', 'Publisher', 'Platform'], inplace = True)

In [210]:
data

,Name,Genre,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Rating
0,Rapala Trophies,Sports,0.04,0.00,0.00,0.00,NaN,NaN,NaN,NaN,E
1,New Super Mario Bros. U,Platform,2.30,1.34,1.27,0.32,84.0,70.0,8.1,733.0,E
2,Robots,Action,0.18,0.14,0.00,0.05,53.0,6.0,6.0,8.0,E
3,Hamster Club 3,Simulation,0.00,0.00,0.35,0.01,NaN,NaN,NaN,NaN,NaN
4,Formula 1 06,Racing,0.00,0.00,0.04,0.00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
11698,Call of Duty: Black Ops,Shooter,9.70,3.68,0.11,1.13,87.0,89.0,6.3,1454.0,M
11699,Resistance 3,Shooter,0.64,0.57,0.05,0.22,83.0,91.0,7.8,543.0,M
11700,Ready 2 Rumble Revolution,Sports,0.15,0.00,0.00,0.01,37.0,31.0,NaN,NaN,T
11701,Monster Hunter Frontier Online: Forward 2,Role-Playing,0.00,0.00,0.03,0.00,NaN,NaN,NaN,NaN,NaN


In [211]:
X = data.iloc[:, :].values; X[0]

array(['Rapala Trophies', 'Sports', 0.04, 0.0, 0.0, 0.0, nan, nan, nan,
       nan, 'E'], dtype=object)

In [212]:
X = np.delete(X, 4, 1); X[0]

array(['Rapala Trophies', 'Sports', 0.04, 0.0, 0.0, nan, nan, nan, nan,
       'E'], dtype=object)

In [213]:
X[0]

array(['Rapala Trophies', 'Sports', 0.04, 0.0, 0.0, nan, nan, nan, nan,
       'E'], dtype=object)

In [214]:
y = data.iloc[:, 4:5].values

In [215]:
y

array([[0.  ],
       [1.27],
       [0.  ],
       ...,
       [0.  ],
       [0.03],
       [0.  ]])

In [216]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [217]:
games_in_training_set = X_train[:, 0]
games_in_test_set = X_test[:, 0]

In [218]:
X_train = X_train[:, 1:]
X_test = X_test[:, 1:]

In [219]:
X_train[0]

array(['Role-Playing', 0.0, 0.0, 0.0, nan, nan, nan, nan, nan],
      dtype=object)

In [220]:
imputer = SimpleImputer(missing_values=np.nan, strategy = 'median')

In [221]:
X_train[:, [4, 5 ,6, 7]] = imputer.fit_transform(X_train[:, [4, 5, 6, 7]])
X_test[:, [4, 5 ,6, 7]] = imputer.transform(X_test[:, [4, 5, 6, 7]])

In [222]:
categorical_imputer = SimpleImputer(strategy = 'most_frequent')

In [223]:
X_train[:, [0, 8]] = categorical_imputer.fit_transform(X_train[:, [0, 8]])
X_test[:, [0, 8]] = categorical_imputer.transform(X_test[:, [0, 8]])

In [224]:
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(handle_unknown='ignore'), [0, 8])], remainder = 'passthrough') 
X_train = ct.fit_transform(X_train)
X_test = ct.transform(X_test)

In [225]:
model = GradientBoostingRegressor(loss = 'absolute_error', n_estimators = 2000, learning_rate= 0.1)

In [226]:
model.fit(X_train, y_train)

C:\Users\nabr9\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor(loss='absolute_error', n_estimators=2000)

In [227]:
y_pred = model.predict(X_test)

In [228]:
games_in_test_set = games_in_test_set.reshape(-1, 1)
y_pred = y_pred.reshape(-1, 1)
predictions = np.concatenate([games_in_test_set, y_pred, y_test], axis = 1)
predictions = pd.DataFrame(predictions, columns = ['Name', 'Predicted_JP_Sales', 'Actual_JP_Sales'])

In [229]:
predictions

,Name,Predicted_JP_Sales,Actual_JP_Sales
0,Cartoon Network Collection: Game Boy Advance V...,0.0,0.0
1,Disney's Kim Possible: Revenge of Monkey Fist,0.0,0.0
2,Imagine: Reporter,0.0,0.0
3,Fade to Black,0.0,0.0
4,High School Musical: Sing It!,0.0,0.0
...,...,...,...
3506,RoadKill,0.0,0.0
3507,Riviera: The Promised Land,0.011045,0.02
3508,Sid Meier's Civilization IV,0.007984,0.0
3509,The Idolmaster: Gravure For You! Vol.8,0.063224,0.02


In [230]:
data2 = pd.read_csv('Video_Games_Test.csv')

In [231]:
data2

,Id,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,Other_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,1,BeatMania IIDX 13: DistorteD,PS2,2007.0,Simulation,Konami Digital Entertainment,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Rugrats: Castle Capers,GBA,2001.0,Action,THQ,0.33,0.12,0.01,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Overlord II,PS3,2009.0,Action,Codemasters,0.11,0.15,0.06,72.0,42.0,7.6,24.0,Triumph Studios,T
3,4,Barbie as The Island Princess,GBA,2007.0,Adventure,Activision,0.16,0.06,0.00,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Petz: Hamsterz Life 2,GBA,2007.0,Misc,Ubisoft,0.01,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5011,5012,Valhalla Knights: Eldar Saga,Wii,2009.0,Role-Playing,Rising Star Games,0.10,0.01,0.01,35.0,17.0,6.7,19.0,"K2, Kurogane",T
5012,5013,WWF No Mercy,N64,2000.0,Fighting,THQ,1.20,0.27,0.02,NaN,NaN,NaN,NaN,NaN,NaN
5013,5014,Ty the Tasmanian Tiger,GC,2002.0,Platform,Electronic Arts,0.21,0.05,0.01,69.0,12.0,8.7,15.0,Krome Studios,E
5014,5015,Scooby-Doo! Unmasked,DS,2005.0,Platform,THQ,0.07,0.01,0.01,NaN,NaN,NaN,NaN,NaN,NaN


In [232]:
ID = data2['Id'].values

In [233]:
ID

array([   1,    2,    3, ..., 5014, 5015, 5016], dtype=int64)

In [234]:
data2.drop(columns = ['Id','Year_of_Release', 'Developer', 'Publisher', 'Platform'], inplace = True)

In [235]:
X = data2.values

In [236]:
X

array([['BeatMania IIDX 13: DistorteD', 'Simulation', 0.0, ..., nan, nan,
        nan],
       ['Rugrats: Castle Capers', 'Action', 0.33, ..., nan, nan, nan],
       ['Overlord II', 'Action', 0.11, ..., 7.6, 24.0, 'T'],
       ...,
       ['Ty the Tasmanian Tiger', 'Platform', 0.21, ..., 8.7, 15.0, 'E'],
       ['Scooby-Doo! Unmasked', 'Platform', 0.07, ..., nan, nan, nan],
       ['From Russia With Love', 'Action', 0.34, ..., 7.6, 36.0, 'T']],
      dtype=object)

In [237]:
X = X[:, 1:]

In [238]:
X[0]

array(['Simulation', 0.0, 0.0, 0.0, nan, nan, nan, nan, nan], dtype=object)

In [239]:
X[:, [4, 5 ,6, 7]] = imputer.transform(X[:, [4, 5, 6, 7]])

In [240]:
X[0]

array(['Simulation', 0.0, 0.0, 0.0, 71.0, 21.0, 7.5, 24.0, nan],
      dtype=object)

In [241]:
X[:, [0, 8]] = categorical_imputer.transform(X[:, [0, 8]])

In [242]:
X[0]

array(['Simulation', 0.0, 0.0, 0.0, 71.0, 21.0, 7.5, 24.0, 'E'],
      dtype=object)

In [243]:
X = ct.transform(X)

In [244]:
X

array([[0.0, 0.0, 0.0, ..., 21.0, 7.5, 24.0],
       [1.0, 0.0, 0.0, ..., 21.0, 7.5, 24.0],
       [1.0, 0.0, 0.0, ..., 42.0, 7.6, 24.0],
       ...,
       [0.0, 0.0, 0.0, ..., 12.0, 8.7, 15.0],
       [0.0, 0.0, 0.0, ..., 21.0, 7.5, 24.0],
       [1.0, 0.0, 0.0, ..., 29.0, 7.6, 36.0]], dtype=object)

In [245]:
y_pred = model.predict(X)

In [246]:
y_pred

array([0.06692937, 0.        , 0.        , ..., 0.        , 0.        ,
       0.00798449])

In [247]:
ID = ID.reshape(-1,1)
y_pred = y_pred.reshape(-1, 1)
predictions = np.concatenate([ID, y_pred], axis = 1)

In [248]:
predictions = pd.DataFrame(predictions, columns = ['Id', 'JP_Sales'])

In [249]:
predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5016 entries, 0 to 5015
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Id        5016 non-null   float64
 1   JP_Sales  5016 non-null   float64
dtypes: float64(2)
memory usage: 78.5 KB


In [250]:
predictions['Id'] = predictions['Id'].astype(int)

In [251]:
predictions.to_csv('sales_result_3.csv', index=False,encoding='utf-8')

In [252]:
loss = ["ls", "lad", "huber"]

max_depth = [2, 3, 5, 10, 15]

min_samples_leaf = [1, 2, 4, 6, 8]

min_samples_split = [2, 4, 6, 10]

max_features = ["auto", "sqrt", "log2", None]

hyperparameter_grid = {"loss": loss,
                       "max_depth": max_depth,
                       "min_samples_leaf": min_samples_leaf,
                       "min_samples_split": min_samples_split,
                       "max_features": max_features}

In [253]:
basic_model = GradientBoostingRegressor(random_state = 42)

random_cv = RandomizedSearchCV(estimator=basic_model,
                               param_distributions=hyperparameter_grid,
                               cv=4, n_iter=20, 
                               scoring="neg_mean_absolute_error",
                               n_jobs=-1, verbose=1, 
                               return_train_score=True,
                               random_state=42)

In [254]:
random_cv.fit(X_train, y_train)

Fitting 4 folds for each of 20 candidates, totalling 80 fits


C:\Users\nabr9\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
56 fits failed out of a total of 80.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
6 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\nabr9\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\nabr9\anaconda3\lib\site-packages\sklearn\base.py", line 1145, in wrapper
    estimator._validate_params()
  File "C:\Users\nabr9\anaconda3\lib\site-packages\sklearn\base.py", line 638, in _validate_params
    validate_parameter_constraints(
  File "C:\Users\nabr9\anaconda3\lib\site-packages\s

RandomizedSearchCV(cv=4, estimator=GradientBoostingRegressor(random_state=42),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'loss': ['ls', 'lad', 'huber'],
                                        'max_depth': [2, 3, 5, 10, 15],
                                        'max_features': ['auto', 'sqrt', 'log2',
                                                         None],
                                        'min_samples_leaf': [1, 2, 4, 6, 8],
                                        'min_samples_split': [2, 4, 6, 10]},
                   random_state=42, return_train_score=True,
                   scoring='neg_mean_absolute_error', verbose=1)

In [255]:
random_results = pd.DataFrame(random_cv.cv_results_).sort_values("mean_test_score", ascending=False)
random_results.head(10)[["mean_test_score", "param_loss",
                         "param_max_depth", "param_min_samples_leaf", "param_min_samples_split",
                         "param_max_features"]]

,mean_test_score,param_loss,param_max_depth,param_min_samples_leaf,param_min_samples_split,param_max_features
0,-0.069980,huber,15,8,6,log2
1,-0.074022,huber,2,2,4,None
10,-0.076292,huber,2,1,2,sqrt
18,-0.076602,huber,2,8,6,sqrt
13,-0.077220,huber,2,4,4,log2
9,-0.077637,huber,2,6,10,log2
2,NaN,ls,3,1,4,sqrt
3,NaN,lad,2,8,10,sqrt
4,NaN,ls,2,8,6,log2
5,NaN,huber,15,1,2,auto


In [256]:
random_cv.best_estimator_

GradientBoostingRegressor(loss='huber', max_depth=15, max_features='log2',
                          min_samples_leaf=8, min_samples_split=6,
                          random_state=42)

In [257]:
from sklearn.model_selection import GridSearchCV
trees_grid = {"n_estimators": [50, 100, 150, 200, 250, 300, 1000, 1500, 2000, 2500]}

basic_model = random_cv.best_estimator_
grid_search = GridSearchCV(estimator=basic_model, param_grid=trees_grid, cv=4, 
                           scoring="neg_mean_absolute_error", verbose=1,
                           n_jobs=-1, return_train_score=True)

In [258]:
grid_search.fit(X_train, y_train)

Fitting 4 folds for each of 10 candidates, totalling 40 fits


C:\Users\nabr9\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv=4,
             estimator=GradientBoostingRegressor(loss='huber', max_depth=15,
                                                 max_features='log2',
                                                 min_samples_leaf=8,
                                                 min_samples_split=6,
                                                 random_state=42),
             n_jobs=-1,
             param_grid={'n_estimators': [50, 100, 150, 200, 250, 300, 1000,
                                          1500, 2000, 2500]},
             return_train_score=True, scoring='neg_mean_absolute_error',
             verbose=1)

In [259]:
basic_final_model = grid_search.best_estimator_
basic_final_model

GradientBoostingRegressor(loss='huber', max_depth=15, max_features='log2',
                          min_samples_leaf=8, min_samples_split=6,
                          random_state=42)

In [260]:
def mae(y_true, y_pred):
    return np.average(abs(y_true - y_pred))

In [261]:
basic_final_pred = basic_final_model.predict(X_test)
basic_final_mae = mae(y_test, basic_final_pred)
print("Final model performance on the test set: MAE = {:.04f}.".format(basic_final_mae))

Final model performance on the test set: MAE = 0.1090.


In [299]:
y_pred = basic_final_model.predict(X)

In [300]:
ID = ID.reshape(-1,1)
y_pred = y_pred.reshape(-1, 1)
predictions = np.concatenate([ID, y_pred], axis = 1)

In [301]:
predictions = pd.DataFrame(predictions, columns = ['Id', 'JP_Sales'])

In [302]:
predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5016 entries, 0 to 5015
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Id        5016 non-null   float64
 1   JP_Sales  5016 non-null   float64
dtypes: float64(2)
memory usage: 78.5 KB


In [303]:
predictions['Id'] = predictions['Id'].astype(int)

In [304]:
predictions

,Id,JP_Sales
0,1,0.064913
1,2,0.007572
2,3,0.001741
3,4,0.004840
4,5,0.003629
...,...,...
5011,5012,0.052403
5012,5013,0.012964
5013,5014,0.005443
5014,5015,0.011365


In [307]:
predictions.to_csv('sales_result_3.csv', index=False,encoding='utf-8')

In [306]:
predictions

,Id,JP_Sales
0,1,0.0650
1,2,0.0076
2,3,0.0018
3,4,0.0049
4,5,0.0037
...,...,...
5011,5012,0.0525
5012,5013,0.0130
5013,5014,0.0055
5014,5015,0.0114
